<a href="https://colab.research.google.com/github/averma12/Beginning_Data_Science/blob/master/Concurrency_I/O.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Sync version of requests
import requests
import time

In [0]:
def download_site(url, session):
    with session.get(url) as response:
        print(f"Read {len(response.content)} from {url}")

In [0]:
def download_all_sites(sites):
    with requests.Session() as session:
        for url in sites:
            download_site(url, session)

In [5]:
 sites = [
        "http://www.jython.org",
        "http://olympus.realpython.org/dice",
    ] * 80
print(len(sites))
start_time = time.time()
download_all_sites(sites)
duration = time.time() - start_time
print(f"Downloaded {len(sites)} in {duration} seconds")

160
Read 19210 from http://www.jython.org
Read 274 from http://olympus.realpython.org/dice
Read 19210 from http://www.jython.org
Read 274 from http://olympus.realpython.org/dice
Read 19210 from http://www.jython.org
Read 274 from http://olympus.realpython.org/dice
Read 19210 from http://www.jython.org
Read 274 from http://olympus.realpython.org/dice
Read 19210 from http://www.jython.org
Read 274 from http://olympus.realpython.org/dice
Read 19210 from http://www.jython.org
Read 274 from http://olympus.realpython.org/dice
Read 19210 from http://www.jython.org
Read 274 from http://olympus.realpython.org/dice
Read 19210 from http://www.jython.org
Read 274 from http://olympus.realpython.org/dice
Read 19210 from http://www.jython.org
Read 274 from http://olympus.realpython.org/dice
Read 19210 from http://www.jython.org
Read 274 from http://olympus.realpython.org/dice
Read 19210 from http://www.jython.org
Read 274 from http://olympus.realpython.org/dice
Read 19210 from http://www.jython.org
R

**19.5 seconds to download . Yikes!!!!!!**

In [0]:
#threading version
import concurrent.futures
import threading

In [0]:
thread_local = threading.local()

In [10]:
thread_local

In [0]:
def get_session():
    if not getattr(thread_local, "session", None):
        thread_local.session = requests.Session()
    return thread_local.session

In [17]:
for att in dir(thread_local):
    print (att, getattr(thread_local,att))

__class__ <class '_thread._local'>
__delattr__ <method-wrapper '__delattr__' of _thread._local object at 0x7fba57a57410>
__dir__ <built-in method __dir__ of _thread._local object at 0x7fba57a57410>
__doc__ Thread-local data
__eq__ <method-wrapper '__eq__' of _thread._local object at 0x7fba57a57410>
__format__ <built-in method __format__ of _thread._local object at 0x7fba57a57410>
__ge__ <method-wrapper '__ge__' of _thread._local object at 0x7fba57a57410>
__getattribute__ <method-wrapper '__getattribute__' of _thread._local object at 0x7fba57a57410>
__gt__ <method-wrapper '__gt__' of _thread._local object at 0x7fba57a57410>
__hash__ <method-wrapper '__hash__' of _thread._local object at 0x7fba57a57410>
__init__ <method-wrapper '__init__' of _thread._local object at 0x7fba57a57410>
__init_subclass__ <built-in method __init_subclass__ of type object at 0xa23bc0>
__le__ <method-wrapper '__le__' of _thread._local object at 0x7fba57a57410>
__lt__ <method-wrapper '__lt__' of _thread._local ob

In [19]:
requests.Session()

In [0]:
def download_site(url):
    session = get_session()
    with session.get(url) as response:
        print(f"Read {len(response.content)} from {url}")


In [0]:
def download_all_sites(sites,workers):
    with concurrent.futures.ThreadPoolExecutor(max_workers=workers) as executor:
        executor.map(download_site, sites)

In [25]:
sites = [
        "http://www.jython.org",
        "http://olympus.realpython.org/dice",
    ] * 80
start_time = time.time()
download_all_sites(sites,20)
duration = time.time() - start_time
print(f"Downloaded {len(sites)} in {duration} seconds")

Read 274 from http://olympus.realpython.org/dice
Read 274 from http://olympus.realpython.org/dice
Read 274 from http://olympus.realpython.org/dice
Read 274 from http://olympus.realpython.org/dice
Read 274 from http://olympus.realpython.org/dice
Read 274 from http://olympus.realpython.org/diceRead 274 from http://olympus.realpython.org/dice

Read 274 from http://olympus.realpython.org/diceRead 274 from http://olympus.realpython.org/diceRead 274 from http://olympus.realpython.org/dice


Read 274 from http://olympus.realpython.org/dice
Read 274 from http://olympus.realpython.org/dice
Read 274 from http://olympus.realpython.org/dice
Read 274 from http://olympus.realpython.org/dice
Read 274 from http://olympus.realpython.org/dice
Read 274 from http://olympus.realpython.org/dice
Read 19210 from http://www.jython.org
Read 19210 from http://www.jython.org
Read 274 from http://olympus.realpython.org/dice
Read 19210 from http://www.jython.org
Read 274 from http://olympus.realpython.org/dice
Read

In [27]:
!pip install aiohttp

    100% |████████████████████████████████| 1.2MB 15.0MB/s 
    100% |████████████████████████████████| 317kB 29.4MB/s 
    100% |████████████████████████████████| 256kB 31.3MB/s 
  Stored in directory: /root/.cache/pip/wheels/d3/00/b3/32d613e19e08a739751dd6bf998cfed277728f8b2127ad4eb7
Successfully built idna-ssl


In [0]:
#Asyncio version
import asyncio
import time
import aiohttp

In [0]:
async def download_site(session, url):
    async with session.get(url) as response:
        print("Read {0} from {1}".format(response.content_length, url))

In [0]:
async def download_all_sites(sites):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for url in sites:
            task = asyncio.ensure_future(download_site(session, url))
            tasks.append(task)
        await asyncio.gather(*tasks, return_exceptions=True)

In [31]:
sites = [
        "http://www.jython.org",
        "http://olympus.realpython.org/dice",
    ] * 80
start_time = time.time()
asyncio.get_event_loop().run_until_complete(download_all_sites(sites))
duration = time.time() - start_time
print(f"Downloaded {len(sites)} sites in {duration} seconds")

Read 274 from http://olympus.realpython.org/dice
Read 274 from http://olympus.realpython.org/dice
Read 274 from http://olympus.realpython.org/dice
Read 274 from http://olympus.realpython.org/dice
Read 274 from http://olympus.realpython.org/dice
Read 274 from http://olympus.realpython.org/dice
Read 274 from http://olympus.realpython.org/dice
Read 274 from http://olympus.realpython.org/dice
Read 274 from http://olympus.realpython.org/dice
Read 274 from http://olympus.realpython.org/dice
Read 274 from http://olympus.realpython.org/dice
Read 274 from http://olympus.realpython.org/dice
Read 274 from http://olympus.realpython.org/dice
Read 274 from http://olympus.realpython.org/dice
Read 274 from http://olympus.realpython.org/dice
Read 274 from http://olympus.realpython.org/dice
Read 274 from http://olympus.realpython.org/dice
Read 274 from http://olympus.realpython.org/dice
Read 274 from http://olympus.realpython.org/dice
Read 274 from http://olympus.realpython.org/dice
Read 274 from http:/

In [0]:
session = None

In [0]:
#multiprocessing
import requests
import multiprocessing
import time

In [34]:
multiprocessing.Pool()

In [0]:
def set_global_session():
    global session
    if not session:
        session = requests.Session()

In [0]:
def download_site(url):
    with session.get(url) as response:
        name = multiprocessing.current_process().name
        print(f"{name}:Read {len(response.content)} from {url}")

In [0]:
def download_all_sites(sites):
    with multiprocessing.Pool(initializer=set_global_session) as pool:
        pool.map(download_site, sites)

In [38]:
start_time = time.time()
download_all_sites(sites)
duration = time.time() - start_time
print(f"Downloaded {len(sites)} in {duration} seconds")

ForkPoolWorker-3:Read 19210 from http://www.jython.org
ForkPoolWorker-4:Read 19210 from http://www.jython.org
ForkPoolWorker-3:Read 274 from http://olympus.realpython.org/dice
ForkPoolWorker-4:Read 274 from http://olympus.realpython.org/dice
ForkPoolWorker-3:Read 19210 from http://www.jython.org
ForkPoolWorker-3:Read 274 from http://olympus.realpython.org/dice
ForkPoolWorker-4:Read 19210 from http://www.jython.org
ForkPoolWorker-3:Read 19210 from http://www.jython.org
ForkPoolWorker-4:Read 274 from http://olympus.realpython.org/dice
ForkPoolWorker-3:Read 274 from http://olympus.realpython.org/dice
ForkPoolWorker-3:Read 19210 from http://www.jython.org
ForkPoolWorker-4:Read 19210 from http://www.jython.org
ForkPoolWorker-3:Read 274 from http://olympus.realpython.org/dice
ForkPoolWorker-4:Read 274 from http://olympus.realpython.org/dice
ForkPoolWorker-3:Read 19210 from http://www.jython.org
ForkPoolWorker-3:Read 274 from http://olympus.realpython.org/dice
ForkPoolWorker-4:Read 19210 from